In [1]:
import pandas as pd
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt

from torch.utils.data import Dataset
from torchvision.transforms import transforms
from torch.utils.data import DataLoader

import rasterio

import warnings
warnings.filterwarnings("ignore")

In [20]:
# Define torch dataset Class
class Dataset(Dataset):
    def __init__(self,folder_path,dataset_file,sen2_amount=1):
        
        # define filepaths
        self.folder_path = folder_path
        # read file
        self.df = pd.read_pickle(dataset_file)
        # set amount of sen2 pictures that should be returned
        self.sen2_amount = sen2_amount
        
        # clear up DF
        self.df = self.df[self.df["sen2_no"]!=0]
        self.df = self.df.reset_index()
        
        # define transformer
        self.transform_sen = transforms.Compose([
            transforms.Normalize(mean=[479.0, 537.0, 344.0], std=[430.0, 290.0, 229.0]) ])
        self.transform_spot = transforms.Compose([
            transforms.Normalize(mean=[78.0, 91.0, 62.0], std=[36.0, 28.0, 30.0]) ])
        
 
    def __len__(self):
        """
        Returns length of data
        """
        return(len(self.df))
    
 
    def __getitem__(self,idx):
        
        current = self.df.iloc[idx]
        spot6_file = current["spot6_filenames"]
        sen2_files = current["sen2_filenames"]
        
        """READ SPOT6"""
        #with rasterio.open(self.folder_path+"y/"+spot6_file) as dataset:
        spot6 = rasterio.open(self.folder_path+"y/"+spot6_file).read()
        #spot6 = raster
    
    
        """READ SEN2 SERIES"""
        # read first file
        sen2 = rasterio.open(self.folder_path+"x/"+sen2_files[0]).read()
        
        if self.sen2_amount>1:
            # read following sen2 and stack
            count=1
            for sen2_file in sen2_files[1:]:
                # read file as array
                sen2_following = rasterio.open(self.folder_path+"x/"+sen2_file).read()
                # stack to previous images
                sen2 = np.concatenate([sen2, sen2_following])

                # break if all wanted files loaded
                count=count+1
                if count==self.sen2_amount:
                    break
        
        return(spot6,sen2)
            
        
    
        """
        # Perform Transform and change types
        im_sen2  = torch.from_numpy(im_sen2)
        im_spot6 = torch.from_numpy(im_spot6)
        
        im_sen2 = im_sen2.float()
        im_spot6 = im_spot6.float()
        
        im_sen2  = self.transform_sen(im_sen2)
        im_spot6 = self.transform_spot(im_spot6)
        """
        #return(im_sen2,im_spot6)

    

In [21]:
dataset = Dataset("data_f4/","df_saved_images.pkl",1)
loader = DataLoader(dataset,batch_size=1, shuffle=True, num_workers=1)

In [26]:
%time
for i in loader:
    a,b = i
print("done!")

CPU times: user 6 µs, sys: 1e+03 ns, total: 7 µs
Wall time: 16 µs
done!


In [ ]:
dataset.df.to_csv("export.csv")

In [23]:
dataset.df.describe()

,level_0,index,x,y,index_right,min,max,sen2_no
count,66499.000000,66499.000000,66499.000000,6.649900e+04,66499.000000,66499.000000,66499.000000,66499.000000
mean,33674.706672,85684.865457,271049.053369,6.801503e+06,1668.955608,2.139175,111.169100,3.392653
std,19710.622090,46037.806905,68479.331806,3.391845e+04,958.077492,3.962137,22.864399,0.908871
min,0.000000,1028.000000,126000.000000,6.709000e+06,0.000000,0.000000,82.000000,1.000000
25%,16624.500000,45416.500000,217500.000000,6.776500e+06,880.500000,0.000000,95.000000,3.000000
50%,33304.000000,85823.000000,267500.000000,6.800500e+06,1659.000000,1.000000,97.000000,4.000000
75%,50625.500000,126414.500000,327000.000000,6.828000e+06,2485.000000,3.000000,135.000000,4.000000
max,68199.000000,162527.000000,401000.000000,6.879000e+06,3351.000000,25.000000,159.000000,4.000000
